<a href="https://colab.research.google.com/github/JosePabloGFnl/Tensorflow-Practice/blob/main/Module6NaturalLanguageProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#----NATURAL LANGUAGE PROCESSING-----
#Natural Language Processing (or NLP for short) is a discipline in computing that deals with the 
#communication between natural (human) languages and computer languages. 
#A common example of NLP is something like spellcheck or autocomplete. Essentially NLP is the field that focuses on how 
#computers can understand and/or process natural/human languages

In [ ]:
#We will learn how to use a reccurent neural network to do the following:

#Sentiment Analysis
#Character Generation

In [ ]:
#BAG OF WORDS TECHNIQUE
#consult notebook for info

vocab = {}  # maps word to integer representing it
word_encoding = 1
def bag_of_words(text):
  global word_encoding

  words = text.lower().split(" ")  # create a list of all of the words in the text, well assume there is no grammar in our text for this example
  bag = {}  # stores all of the encodings and their frequency

  for word in words:
    if word in vocab:
      encoding = vocab[word]  # get encoding from vocab
    else:
      vocab[word] = word_encoding
      encoding = word_encoding
      word_encoding += 1
    
    if encoding in bag:
      bag[encoding] += 1
    else:
      bag[encoding] = 1
  
  return bag

text = "this is a test to see if this test will work is is test a a"
bag = bag_of_words(text)
print(bag)
print(vocab)

{1: 2, 2: 3, 3: 3, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


In [ ]:
#-Sentiment Analysis-
#the process of computationally identifying and categorizing opinions expressed in a piece of text, 
#especially in order to determine whether the writer's attitude towards a particular topic, product, etc. is positive, negative, or neutral.

#The example we’ll use here is classifying movie reviews as either postive, negative or neutral. with already preprocessed reviews

%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584 #the 88584 word is the least common in the data set

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.
17473536/17464789 [==============================] - 0s 0us/step


In [ ]:
# Lets look at one review
train_data[1]
#it will print all of the words in the review

[1,
 194,
 1153,
 194,
 8255,
 78,
 228,
 5,
 6,
 1463,
 4369,
 5012,
 134,
 26,
 4,
 715,
 8,
 118,
 1634,
 14,
 394,
 20,
 13,
 119,
 954,
 189,
 102,
 5,
 207,
 110,
 3103,
 21,
 14,
 69,
 188,
 8,
 30,
 23,
 7,
 4,
 249,
 126,
 93,
 4,
 114,
 9,
 2300,
 1523,
 5,
 647,
 4,
 116,
 9,
 35,
 8163,
 4,
 229,
 9,
 340,
 1322,
 4,
 118,
 9,
 4,
 130,
 4901,
 19,
 4,
 1002,
 5,
 89,
 29,
 952,
 46,
 37,
 4,
 455,
 9,
 45,
 43,
 38,
 1543,
 1905,
 398,
 4,
 1649,
 26,
 6853,
 5,
 163,
 11,
 3215,
 10156,
 4,
 1153,
 9,
 194,
 775,
 7,
 8255,
 11596,
 349,
 2637,
 148,
 605,
 15358,
 8003,
 15,
 123,
 125,
 68,
 23141,
 6853,
 15,
 349,
 165,
 4362,
 98,
 5,
 4,
 228,
 9,
 43,
 36893,
 1157,
 15,
 299,
 120,
 5,
 120,
 174,
 11,
 220,
 175,
 136,
 50,
 9,
 4373,
 228,
 8255,
 5,
 25249,
 656,
 245,
 2350,
 5,
 4,
 9837,
 131,
 152,
 491,
 18,
 46151,
 32,
 7464,
 1212,
 14,
 9,
 6,
 371,
 78,
 22,
 625,
 64,
 1382,
 9,
 8,
 168,
 145,
 23,
 4,
 1690,
 15,
 16,
 4,
 1355,
 5,
 28,
 6,
 52,
 

In [ ]:
#our reviews are of different length, this is an issue, we must have them all of one same length
#we will pad our sequences, if the review is greater than 250 words we will trim some extra words, on the contrary we will add

train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [ ]:
train_data[1] #the 0's are due to the padding we applied

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     1,   194,
        1153,   194,  8255,    78,   228,     5,     6,  1463,  4369,
        5012,   134,    26,     4,   715,     8,   118,  1634,    14,
         394,    20,    13,   119,   954,   189,   102,     5,   207,
         110,  3103,    21,    14,    69,   188,     8,    30,    23,
           7,     4,   249,   126,    93,     4,   114,     9,  2300,
        1523,     5,   647,     4,   116,     9,    35,  8163,     4,
         229,     9,   340,  1322,     4,   118,     9,     4,   130,
        4901,    19,

In [ ]:
#creating the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#training
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

#to accelerate the training: runtime > change > hardware accelerator gpu

Epoch 1/10
625/625 [==============================] - 12s 11ms/step - loss: 0.4310 - acc: 0.7966 - val_loss: 0.3397 - val_acc: 0.8558
Epoch 2/10
625/625 [==============================] - 7s 11ms/step - loss: 0.2435 - acc: 0.9078 - val_loss: 0.3025 - val_acc: 0.8828
Epoch 3/10
625/625 [==============================] - 7s 11ms/step - loss: 0.1830 - acc: 0.9329 - val_loss: 0.2782 - val_acc: 0.8956
Epoch 4/10
625/625 [==============================] - 7s 11ms/step - loss: 0.1506 - acc: 0.9449 - val_loss: 0.3150 - val_acc: 0.8918
Epoch 5/10
625/625 [==============================] - 7s 11ms/step - loss: 0.1259 - acc: 0.9546 - val_loss: 0.4253 - val_acc: 0.8454
Epoch 6/10
625/625 [==============================] - 7s 11ms/step - loss: 0.1093 - acc: 0.9612 - val_loss: 0.3152 - val_acc: 0.8892
Epoch 7/10
625/625 [==============================] - 7s 11ms/step - loss: 0.0965 - acc: 0.9685 - val_loss: 0.3186 - val_acc: 0.8744
Epoch 8/10
625/625 [==============================] - 7s 11ms/step -

In [ ]:
#NOTE: Look for optimizers, like adam or rmsprop

In [ ]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 4s 5ms/step - loss: 0.4457 - acc: 0.8475
[0.4456653594970703, 0.8475199937820435]


In [ ]:
#Making predictions

#now that we trained our model, we will use it to make predictions on our own reviews.
#Since our reviews are encoded well need to convert any review that we write into that form so the network can understand it. 
#To do that well load the encodings from the dataset and use them to encode our own data.

word_index = imdb.get_word_index() #we can get the word index and print it

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

1654784/1641221 [==============================] - 0s 0us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

In [ ]:
# while were at it lets make a decode function

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]
  
print(decode_integers(encoded))

#the encoded text gets decoded

that movie was just amazing so amazing


In [ ]:
# now time to make a prediction

def predict(text):
  encoded_text = encode_text(text) #we encode the text with the function above
  pred = np.zeros((1,250)) #we create a blank numpy array in 0's, the shape that our model expects is at max 250
  pred[0] = encoded_text #we insert our entry into the array
  result = model.predict(pred) #we model the array
  print(result[0])

positive_review = "That movie was so awesome! I really loved it and would watch it again because it was amazingly great" #if we change a word, the difference of % will be significant
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

[0.8169493]
[0.2553257]


In [ ]:
#---RNN Play Generator---

#this is the first NN that creates something for us, it will predict the next character in a sequence
#we give it a sequence as an input and it will predict the next character
#we train the model with a bunch of sequences from Romeo & Juliet

%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1130496/1115394 [==============================] - 0s 0us/step


In [ ]:
#we can load our own text data from out computer, not neccesary in this example
from google.colab import files 
path_to_file = list(files.upload().keys())[0]

IndexError: ignored

In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [ ]:
#encoding
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [ ]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [ ]:
#to convert numeric values into text
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [ ]:
#creating training examples

#the traning examples have input values of some length, we pick the length and then the output will be the exact same shifted by one character

seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int) #converting the string dataset into characters

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [ ]:
#Example of how it works
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [ ]:
#training batches
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
#Building the model

#this function returns a built model
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (64, None, 256)           16640     
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_1 (Dense)             (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Creating a loss function to compile our model

#the input to our model is of length 64, we feed it 64 training examples every time
#we pass 64 entries of length 100 into the model as its training depth

#we can use the model with the parameters that we saved, expecting a different input shape

for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [ ]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-3.76761938e-03 -3.07332980e-03 -3.76729621e-03 ... -4.00916114e-03
    5.18335216e-03  1.26092345e-03]
  [-5.99067286e-03 -7.62989512e-04 -1.82354823e-03 ...  1.72426004e-03
    6.11081440e-03  1.20752270e-03]
  [-7.99202640e-03 -5.46373380e-03  1.03354244e-03 ...  5.12627419e-03
    3.27634439e-03  6.92507951e-03]
  ...
  [ 6.61641592e-03 -3.18826106e-03  2.71500507e-03 ...  5.24223363e-03
   -5.73351514e-03  4.15320741e-03]
  [ 6.34859130e-03 -2.05716887e-03  2.41310592e-03 ...  1.49344560e-02
   -7.77734257e-03 -9.99107491e-04]
  [ 8.29135999e-03  3.31534026e-03 -6.78512035e-04 ...  1.21003063e-02
   -1.19398562e-02 -2.35412712e-03]]

 [[-5.41017950e-03 -2.01345491e-03 -2.83408095e-03 ...  1.02606369e-04
    1.32294116e-03 -5.61666675e-05]
  [-2.95703241e-04 -3.83133581e-03 -2.16791546e-03 ...  6.62889739e-04
   -7.55810831e-03 -3.52275372e-03]
  [ 1.77477207e-03 -1.34114898e-03 -1.08763902e-03 ...  1.03139430e-02
   -7.96142220e-03 -7.33126793e-03]
  ...
  [ 3.890

In [ ]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-0.00376762 -0.00307333 -0.0037673  ... -0.00400916  0.00518335
   0.00126092]
 [-0.00599067 -0.00076299 -0.00182355 ...  0.00172426  0.00611081
   0.00120752]
 [-0.00799203 -0.00546373  0.00103354 ...  0.00512627  0.00327634
   0.00692508]
 ...
 [ 0.00661642 -0.00318826  0.00271501 ...  0.00524223 -0.00573352
   0.00415321]
 [ 0.00634859 -0.00205717  0.00241311 ...  0.01493446 -0.00777734
  -0.00099911]
 [ 0.00829136  0.00331534 -0.00067851 ...  0.01210031 -0.01193986
  -0.00235413]], shape=(100, 65), dtype=float32)


In [ ]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next
#the output is the probability of every charachter in the next text

65
tf.Tensor(
[-3.7676194e-03 -3.0733298e-03 -3.7672962e-03  3.6641404e-03
 -4.3502627e-03 -5.1445952e-03  2.5555445e-04 -1.2329968e-03
  2.4423338e-03  3.8808556e-03  3.7810891e-03  2.3661279e-03
 -4.9884301e-03  4.0323404e-03 -4.1584792e-03  2.2174872e-03
 -6.0416795e-03  9.1636612e-04  3.4224296e-03 -3.9064907e-04
 -2.0469795e-04  1.2818913e-03 -2.9246090e-03 -2.2688450e-04
  3.7710308e-03 -8.1764709e-04  2.1036642e-03  1.3975715e-03
 -4.5382045e-03  3.0454807e-03  3.6413237e-03  2.7848615e-03
  1.7147981e-04 -2.0448768e-03  2.7754391e-03  1.4950894e-03
 -4.3601356e-03 -6.3292717e-04 -4.8892183e-04  2.5127127e-04
  6.8581628e-04 -5.0950167e-04 -9.0326800e-04  4.9525634e-03
 -4.2946122e-04  2.4405064e-03  6.7051128e-04 -4.3317676e-05
  2.4606911e-03 -2.4021755e-03 -4.4638822e-03 -4.8655267e-03
  1.3539928e-03 -2.2099223e-03 -8.2483393e-04 -4.9878294e-03
 -5.7647452e-03 -3.4344546e-03  9.7011626e-03 -1.8916623e-03
  7.0176842e-03 -2.1972321e-03 -4.0091611e-03  5.1833522e-03
  1.260923

In [ ]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

#the output is all of the predicted characters

"CTfGUzOrNKNttwbOOgh\nvt:y\n,!ayMsbw\nau-bHAk'elgUhLSqH$JVAS:FgsiMw:a3A:lZLc'!l?yBiw!fZGamOs.WBAs'w'BunH"

In [ ]:
#this takes all of the labels and logits (probability distributions) and computes a loss

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
#---Compiling the Model---

model.compile(optimizer='adam', loss=loss)

In [ ]:
#---Creating Checkpoints
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
#---Training---
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 14s 67ms/step - loss: 2.5751
Epoch 2/50
172/172 [==============================] - 13s 68ms/step - loss: 1.8714
Epoch 3/50
172/172 [==============================] - 13s 68ms/step - loss: 1.6267
Epoch 4/50
172/172 [==============================] - 13s 67ms/step - loss: 1.4966
Epoch 5/50
172/172 [==============================] - 13s 68ms/step - loss: 1.4181
Epoch 6/50
172/172 [==============================] - 13s 67ms/step - loss: 1.3619
Epoch 7/50
172/172 [==============================] - 13s 67ms/step - loss: 1.3164
Epoch 8/50
172/172 [==============================] - 13s 68ms/step - loss: 1.2787
Epoch 9/50
172/172 [==============================] - 13s 67ms/step - loss: 1.2430
Epoch 10/50
172/172 [==============================] - 13s 68ms/step - loss: 1.2083
Epoch 11/50
172/172 [==============================] - 13s 68ms/step - loss: 1.1727
Epoch 12/50
172/172 [==============================] - 13s 67ms/step - loss: 1.1381
E

In [ ]:
#the more epochs, the better

In [ ]:
#---Loading the model---

#we rebuild the model with a batch size of 1
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [ ]:
#we load the weights
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
#we can load any checkpoint

checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

AttributeError: ignored

In [ ]:
#---Generating the text---

#stolen from tensorflow

#now we can use the tensorflow provided function
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: romeo
romeould have all kindness
And in their woes from breath, some day disged
the heart of the music of the house of Lancaster.
Your brother's son: hang him, and stand on sighs;
But shall wo wive it shameless.

LEONTES:
Say you?

Volsce:
Peter.

MENENIUS:
Be else with me to this beard, than they--
Myselves would offend along.

Provost:
Now, kiss me nobly
To o'er-rough from me, and your emils have retired o' the foot;
His eyes do prove a lady and more wife in honey.

ROMEO:
Why, she's your highness,--

SICINIUS:
He shall be of noble Pear i' the wars
Since have but truth to thee well see, did vengeance for it!
Shall I be master for his sword: he has, look'd upon:
Come, peace, my lord, my gracious lords.

EDWARD:
But now you know the cause why music were as free to do; and, with a one and such provost
